In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
25,application_1607949680860_0027,pyspark,idle,Link,Link


SparkSession available as 'spark'.

## Import modules

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
import hsfs
from hops import hdfs

## Define utility functions

In [3]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

def extract_node_type(x):
    if (x.startswith("C")):
        node_type = 0
    elif (x.startswith("B")):
        node_type = 1
    elif (x.startswith("M")):
        node_type = 2
    else:
        node_type = 99
    return node_type

def extract_fraudster(x):
    if (x.startswith("CF")):
        fraudster = 1
    else:
        fraudster = 0
    return fraudster

def action_2_code(x):
    if (x == "CASH_IN"):
        node_type = 0
    elif (x == "CASH_OUT"):
        node_type = 1
    elif (x == "DEBIT"):
        node_type = 2
    elif (x == "PAYMENT"):
        node_type = 3
    elif (x == "TRANSFER"):
        node_type = 4
    elif (x == "DEPOSIT"):
        node_type = 4        
    else:
        node_type = 99
    return node_type

def gender_2_code(x):
    if x == 'W':
        gender =  0
    elif x == 'M':
        gender = 1
    else: 
        gender = 99
    return gender

def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    return dt_obj.timestamp()

hashnode_udf = F.udf(hashnode)
extract_fraudster_udf = F.udf(extract_fraudster)
node_type_udf = F.udf(extract_node_type)
action_2_code_udf = F.udf(action_2_code)
gender_2_code_udf = F.udf(gender_2_code)
timestamp_2_time_udf = F.udf(timestamp_2_time)

## Create a connection to Hopsworks feature store (hsfs)

In [4]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Load accounts datasets as spark dataframe and perform feature engineering

In [5]:
accounts_df = spark.read\
             .option("inferSchema", "true")\
             .option("header", "true")\
             .format("csv")\
             .load("hdfs:///Projects/{}/Resources/accounts.csv".format(hdfs.project_name()))

accounts_df = accounts_df.drop('first_name')\
                        .drop('last_name')\
                        .drop('street_addr')\
                        .drop('city')\
                        .drop('state')\
                        .drop('zip')\
                        .drop('gender')\
                        .drop('birth_date')\
                        .drop('ssn')\
                        .drop('lon')\
                        .drop('lat')

In [6]:
accounts_df = accounts_df.withColumn('prior_sar',F.when(F.col('prior_sar_count') == 'true', 1).otherwise(0))\
                         .drop("prior_sar_count","acct_rptng_crncy","type","acct_stat","open_dt","bank_id","country","close_dt","dsply_nm","branch_id")

## Create accounts feature group metadata and save it in to hsfs 

In [7]:
accounts_fg = fs.create_feature_group(name="account_features",
                                      version=1,
                                      primary_key=["acct_id"],
                                      description="node features",
                                      time_travel_format=None,
                                      statistics_config=False)
accounts_fg.save(accounts_df)

## Load transactions datasets as spark dataframe and perform feature engineering

In [8]:
transactions_df = spark.read\
             .option("inferSchema", "true")\
             .option("header", "true")\
             .format("csv")\
             .load("hdfs:///Projects/{}/Resources/transactions.csv".format(hdfs.project_name()))


In [9]:
transactions_df = transactions_df.withColumn('is_sar',F.when(F.col('is_sar') == 'true', 1).otherwise(0))\
                                 .withColumn('tx_type', action_2_code_udf(F.col('tx_type')))\
                                 .withColumn('tran_timestamp', timestamp_2_time_udf(F.col('tran_timestamp')).cast(FloatType()))\
                                 .withColumnRenamed("orig_acct","source")\
                                 .withColumnRenamed("bene_acct","target")\
                                 .select("source","target","tran_timestamp","is_sar","alert_id","tran_id","tx_type","base_amt")
transactions_df.show()

+------+------+--------------+------+--------+-------+-------+--------+
|source|target|tran_timestamp|is_sar|alert_id|tran_id|tx_type|base_amt|
+------+------+--------------+------+--------+-------+-------+--------+
|  1767|  3259|   1.4832288E9|     0|      -1|      1|      4| 9405.71|
|  7363|  5141|   1.4832288E9|     0|      -1|      2|      4| 6884.54|
|  7585|  9532|   1.4832288E9|     0|      -1|      3|      4|  7968.4|
|  1750|  8792|   1.4832288E9|     0|      -1|      4|      4| 9042.67|
|  9060|  4670|   1.4832288E9|     0|      -1|      5|      4| 4692.79|
|  8752|  3861|   1.4832288E9|     0|      -1|      6|      4| 4089.65|
|  9645|  3805|   1.4832288E9|     0|      -1|      7|      4| 3055.04|
|  7150|  9280|   1.4832288E9|     0|      -1|      8|      4| 6473.45|
|  1685|  6369|   1.4832288E9|     0|      -1|      9|      4| 2583.42|
|  7846|  8255|   1.4832288E9|     0|      -1|     10|      4| 6753.04|
|   878|  5957|   1.4832288E9|     0|      -1|     11|      4| 1

## Create transactions feature group metadata and save it in to hsfs 

In [10]:
transactions_fg = fs.create_feature_group(name="transactions_fg",
                                       version=1,
                                       primary_key=["tran_id"],
                                       description="edge features",
                                       time_travel_format=None,                                        
                                       statistics_config=False)
transactions_fg.save(transactions_df)